# Limpeza dos dados

Neste notebook estão algumas operações para a limpeza dos dados obtidos.

Global Biotic Interactions - Os dados limpos para as interações podem ser encontrados nesse [link](https://drive.google.com/file/d/1YhU8QPXXilg4icozIvttXu2hyfm_0yOC/view?usp=sharing)

Red List - Os dados limpos podem ser encontrados nesse [link](https://drive.google.com/file/d/19c4ObCxYTxyAP99bOWejaXKKrvbN8_3M/view?usp=sharing)

## Interações (Global Biotic Interactions)

Esses dados são fornecidos na forma de um arquivo csv contendo em forma de pares as interações entre diferentes seres no meio ambiente. Como o arquivo fornecido inicialmente era muito grande (aproximadamente 14 GB) para possibilitar o manuseio de forma prática, o primeiro passo adotado foi excluir algumas colunas extras contidas no arquivo, mantendo somente os dados referentes aos animais e o tipo de interação que havia entre eles. Para isso, foi utilizada a ferramenta `miller` extrair somente as colunas desejadas do csv. Com o arquivo reduzido para conter somente as informações necessárias, é possível carregar os dados resultantes da seguinte maneira:

In [ ]:
import pandas as pd

interactions_df = pd.read_csv('../../interactions.csv')
interactions_df

No entanto, esses dados ainda contém linhas com informações faltantes, o que pode não ser tão útil. Então, para limpar essa tabela foram removidas todas as linhas contendo dados faltantes nas colunas que indicam o nome científico dos animais que participam da interação e a classificação da interação que ocorre entre eles.

In [ ]:
interactions_df = interactions_df.dropna(subset=['sourceTaxonSpeciesName', 'interactionTypeName', 'targetTaxonSpeciesName'])
interactions_df

In [ ]:
interactions_df.info()

O nome das espécies e dos reinos nem sempre é escrito da mesma forma e as vezes contém erros, como espaços duplos, então é preciso corrigí-los.

In [ ]:
import re
def normalize_species_name(s):
    s = re.sub('\(.*\)', '', s) # Remove parentese
    s = s.lower() # Passa pra minusculo
    s = s.replace('[?�]', '') # Remove ?
    s = re.sub('\s+', ' ', s) # Usa sempre apenas 1 espaços
    s = re.sub('= .* ', '', s) # Remove termos da forma =PALAVRA 
    s = re.sub('= .*,', ',', s) # Remove termos da forma =PALAVRA,
    s = s.strip() # Remove espaços no começo e fim
    return s
interactions_df.sourceTaxonSpeciesName = interactions_df.sourceTaxonSpeciesName.apply(normalize_species_name)
interactions_df.targetTaxonSpeciesName = interactions_df.targetTaxonSpeciesName.apply(normalize_species_name)
interactions_df

Além disso, os dados obtidos podem ter ocorrências repetidas. Para evitar recontar uma mesma interação entre dois animais, serão removidas todas as interações que contenham a mesma fonte, o mesmo alvo e a mesma ação.

In [ ]:
interactions_df = interactions_df.drop_duplicates(subset=['sourceTaxonSpeciesName', 'interactionTypeName', 'targetTaxonSpeciesName'], keep="first")
interactions_df

Outro ponto a se atentar é que algumas espécies são marcadas como **não identificadas**. Como isso não contribuirá diretamente para a análise, interações com espécies não identificadas serão removidas.

In [ ]:
unidentified_source = interactions_df.sourceTaxonSpeciesName.str.contains('unidentified')
interactions_df = interactions_df[~unidentified_source]

unknown_source = interactions_df.sourceTaxonSpeciesName.str.contains('unknown')
interactions_df = interactions_df[~unknown_source]

na_source = interactions_df.sourceTaxonSpeciesName.str.contains('n/a n/a')
interactions_df = interactions_df[~na_source]

num_no_source = sum(map(sum, [unidentified_source, unknown_source, na_source]))
print('Total de interações onde a fonte não foi identificada:', num_no_source)


unidentified_target = interactions_df.targetTaxonSpeciesName.str.contains('unidentified')
interactions_df = interactions_df[~unidentified_target]

unknown_target = interactions_df.targetTaxonSpeciesName.str.contains('unknown')
interactions_df = interactions_df[~unknown_target]

na_target = interactions_df.targetTaxonSpeciesName.str.contains('n/a n/a')
interactions_df = interactions_df[~na_target]

num_no_target = sum(map(sum, [unidentified_target, unknown_target, na_target]))
print('Total de interações onde o alvo não foi identificado:', num_no_target)

interactions_df

Algumas espécies têm registradas interações com elas mesmas, mas essas interações não contribuem para nossa análise e são removidas.

In [ ]:
self_interactions = interactions_df.query('sourceTaxonSpeciesName == targetTaxonSpeciesName')
print("Total de interações com a mesma espécie: {}".format(len(self_interactions)))

interactions_df = interactions_df.drop(self_interactions.index)
interactions_df

Por fim, os dados são salvos mais uma vez em disco.

In [ ]:
interactions_df.to_csv('../../clean-interactions.csv', index=False)

## Red list (IUCN)

Para os dados da Red List, as informações sobre a condição das espécies e as suas localizações foram obtidas de forma separada. Assim, o primeiro passo foi carregar os dados sobre a condição das populações de cada espécie e excluir as colunas com dados que não serão utilizados, como versão do critério ou a língua na qual o critério foi publicado.

In [ ]:
import pandas as pd

redlist_df = pd.read_csv('../../redlist_species_data_83aad1b1-09d6-4283-b629-0ccacb10797b/assessments.csv')
redlist_df = redlist_df[['scientificName', 'redlistCategory', 'populationTrend', 'systems', 'realm', 'possiblyExtinct', 'possiblyExtinctInTheWild']]
redlist_df

Em seguida, carregaram-se os dados de localização das espécies, de onde foram extraídas as colunas contendo o nome científico das espécies e a latitude e longitude da sua localização. Como algumas espécies podem ser encontradas em mais de uma posição, todas as coordenadas foram mantidas na forma de uma lista.

In [ ]:
locations_df = pd.read_csv('../../redlist_species_data_919f6e77-bd6b-4134-acc2-b615e7170293/points_data.csv')
locations_df = locations_df[['binomial', 'longitude', 'latitude']]
locations_df = locations_df.groupby(by='binomial').agg(lambda x: list(x))
locations_df

Após isso, a informação da localização das espécies foi combinada ao restante da tabela, resultando no seguinte:

In [ ]:
redlist_df = redlist_df.join(locations_df, on='scientificName')
redlist_df

Por fim, a tabela com os dados organizados foi salva novamente em disco.

In [ ]:
redlist_df.to_csv('../../clean-redlist.csv')